In [28]:
import pandas as pd
import numpy as np
import copy as cp

Brand options: 'Aramis', 'Aveda', 'Bobbi Brown', 'Bumble and bumble', 'By Kilian',
       'Clinique', 'Darphin', 'Estée Lauder', 'GlamGlow',
       'Jo Malone London', 'La Mer', 'Le Labo', 'M.A.C', 'Origins',
       'Prescriptives', 'Smashbox', 'Tom Ford Beauty'

In [29]:
BRAND = None ####set to none to make dataset for all brands with brand target encoding

## Import and Merge Data

In [30]:
onedrive_path = "C:/Users/cfowle/The Estée Lauder Companies Inc/TeamAnis - General/"

In [32]:
reviews = pd.read_csv(onedrive_path + "Data/Ratings and Reviews/reviews_demand_subcat.csv")
cc      = pd.read_csv(onedrive_path + "Data/Consumer Care/cc_product_month.csv")

C:\Users\cfowle\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
products = pd.read_csv(onedrive_path + "Data/Product/product_codes.csv")

C:\Users\cfowle\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9,12,23,26,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
print(len(reviews))
reviews = reviews.drop(["item_description"], axis = 1).drop_duplicates()
print(len(reviews))

136630
122187


In [35]:
cc = cc.rename(columns = {"Date Month": "date", "P4": "itemid_4", "Brand Clean": "elc_brand"})
products = products.rename(columns = {"P4": "itemid_4", "brand": "elc_brand", "SubCategory":"sub_category"})
products = products[["elc_brand", "itemid_4", 'Major Category', 'Application', 'Category',
       'sub_category']].drop_duplicates(["elc_brand", "itemid_4"])

In [36]:
cc_product = cc.merge(products, how = "left", on = ["itemid_4", "elc_brand"])

In [37]:
cc_reviews_product = reviews.merge(cc_product, how = "left").fillna(0)
cc_reviews_product = cc_reviews_product.groupby(["elc_brand", "date", "sub_category"]).sum().reset_index()

In [38]:
cc_reviews_product = cc_reviews_product.sort_values("date")

In [49]:
cc_reviews_product.to_csv(onedrive_path + "Output/2020_07_06_to_rama/counts.csv")

In [50]:
cc_reviews_product

,elc_brand,date,sub_category,major_category_id,avg_nb_statements,nb_reviews,rating,rating_1,rating_2,rating_3,...,Questions_Service_Miscellaneous,Questions_Website_Offer Codes,Questions_Website_Website functionality,Suggestions_Suggestion_Miscellaneous,avg_subcat_demand,avg_brand_demand,month,year,monthly_avg_demand,avg_12_month_demand
0,Aramis,2014-12,After Shave,530.0,0.000000,0,0.000000,0,0,0,...,0.0,0.0,0.0,0.0,NaN,NaN,12,2014,25517.980220,33970.711538
1,M.A.C,2014-12,All Other Face Makeu,2550.0,0.000000,0,0.000000,0,0,0,...,0.0,0.0,0.0,0.0,NaN,NaN,12,2014,25517.980220,33970.711538
2,M.A.C,2014-12,All Other Foundation,510.0,0.333333,1,0.333333,0,0,1,...,0.0,0.0,0.0,0.0,NaN,NaN,12,2014,25517.980220,33970.711538
3,M.A.C,2014-12,All Other Lip Treatm,520.0,4.500000,1,2.500000,0,0,0,...,0.0,0.0,0.0,0.0,NaN,NaN,12,2014,25517.980220,33970.711538
4,M.A.C,2014-12,All Other Mascara,9180.0,19.115873,33,19.217460,3,0,0,...,0.0,0.0,0.0,0.0,NaN,NaN,12,2014,25517.980220,33970.711538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29570,M.A.C,2020-04,All All Other Eye Ma,510.0,0.000000,0,0.000000,0,0,0,...,0.0,0.0,0.0,0.0,712.500000,3815.833333,04,2020,6170.356044,27236.066300
29571,M.A.C,2020-04,,1530.0,0.000000,0,0.000000,0,0,0,...,0.0,0.0,0.0,0.0,2240.500000,3841.333333,04,2020,6170.356044,27236.066300
29572,Aveda,2020-04,All All Other Eye Ma,510.0,0.000000,0,0.000000,0,0,0,...,0.0,0.0,0.0,0.0,712.500000,3652.833333,04,2020,6170.356044,27236.066300
29573,Estée Lauder,2020-04,All Cleansers (Incl,5200.0,13.262716,44,15.345509,1,4,1,...,0.0,0.0,0.0,0.0,14607.666667,680.333333,04,2020,6170.356044,27236.066300


In [11]:
if BRAND != None:
    cc_reviews_product = cc_reviews_product.loc[cc_reviews_product["elc_brand"] == BRAND]

## Feature Engineering

### Target Encoding

In [43]:
subcat_avg = cc_reviews_product.groupby("sub_category")["demand"].rolling(6).mean().reset_index()
subcat_avg = subcat_avg.set_index('level_1')
cc_reviews_product["avg_subcat_demand"] = subcat_avg['demand']

In [44]:
if BRAND == None:
    brand_avg = cc_reviews_product.groupby("elc_brand")["demand"].rolling(6).mean().reset_index()
    brand_avg = brand_avg.set_index('level_1')
    cc_reviews_product["avg_brand_demand"] = brand_avg['demand']

In [45]:
cc_reviews_product["month"] = [x[-2:] for x in cc_reviews_product["date"].values]
cc_reviews_product["year"]  = [x[:4] for x in cc_reviews_product["date"].values]

In [46]:
month_map = cc_reviews_product.groupby(["month", "year"])["demand"].mean().reset_index()
month_map = month_map.rename(columns = {"demand": "monthly_avg_demand"})
month_map["avg_12_month_demand"] = month_map.rolling(12).mean()["monthly_avg_demand"]

In [47]:
cc_reviews_product = cc_reviews_product.merge(month_map)

In [17]:
if BRAND == None:
    non_customer_data = cc_reviews_product[["demand_F1", "demand", "demand_P2", "demand_P1",
                                   "avg_subcat_demand", "monthly_avg_demand", "avg_12_month_demand", "avg_brand_demand",
                                   "month", "year"]]
else: 
    non_customer_data = cc_reviews_product[["demand_F1", "demand", "demand_P2", "demand_P1",
                                   "avg_subcat_demand", "monthly_avg_demand", "avg_12_month_demand", "month", "year"]]

In [18]:
relative_data = cp.deepcopy(non_customer_data)
subcat = cc_reviews_product ##just here so I can copy some old code easily

In [19]:
relative_data["percent_1"] = subcat["rating_1"]/subcat["nb_reviews"]
relative_data["percent_2"] = subcat["rating_2"]/subcat["nb_reviews"]
relative_data["percent_3"] = subcat["rating_3"]/subcat["nb_reviews"]
relative_data["percent_4"] = subcat["rating_4"]/subcat["nb_reviews"]
relative_data["percent_5"] = subcat["rating_5"]/subcat["nb_reviews"]

In [20]:
relative_data["percent_negative"] = subcat['sentiment_negative']/subcat["nb_reviews"]
relative_data["percent_neutral"] = subcat['sentiment_neutral']/subcat["nb_reviews"]
relative_data["percent_positive"] = subcat['sentiment_positive']/subcat["nb_reviews"]

In [21]:
relative_data = relative_data.fillna(0)

In [22]:
relative_data

,demand_F1,demand,demand_P2,demand_P1,avg_subcat_demand,monthly_avg_demand,avg_12_month_demand,month,year,percent_1,percent_2,percent_3,percent_4,percent_5,percent_negative,percent_neutral,percent_positive
0,0.0,0,0.0,0.0,0.000000,1619.631579,2590.429825,12,2014,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,971.0,723,787.0,865.0,0.000000,1619.631579,2590.429825,12,2014,0.0,0.000000,0.0,0.000000,1.000000,0.000000,0.100000,0.900000
2,194.0,295,261.0,312.0,0.000000,1619.631579,2590.429825,12,2014,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,24018.0,41253,25248.0,60231.0,0.000000,1619.631579,2590.429825,12,2014,0.0,0.000000,0.0,0.181818,0.818182,0.011570,0.243946,0.744484
4,0.0,0,0.0,0.0,0.000000,1619.631579,2590.429825,12,2014,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2465,177.0,77,1135.0,785.0,928.333333,442.184211,2397.846491,04,2020,0.0,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,1.000000
2466,2711.0,4175,16352.0,7570.0,9885.000000,442.184211,2397.846491,04,2020,0.0,0.051282,0.0,0.076923,0.871795,0.058081,0.127600,0.814319
2467,963.0,147,3324.0,645.0,2058.333333,442.184211,2397.846491,04,2020,0.0,0.000000,0.0,0.000000,1.000000,0.000000,0.333333,0.666667
2468,852.0,664,2135.0,1799.0,2334.666667,442.184211,2397.846491,04,2020,0.0,0.000000,0.0,0.400000,0.600000,0.000000,0.070000,0.930000


In [23]:
if BRAND == None:
    relative_data.to_csv(onedrive_path + "Data/Combined/relative_rr_data_w_target_encodings.csv")
else:
    relative_data.to_csv(onedrive_path + "Data/Combined/relative_rr_data_w_target_encodings_" + BRAND + ".csv")

### Relative CC Data

In [24]:
full_relative_data = cp.deepcopy(relative_data)
subcat["cc_tot"] = subcat['Complaints'] + subcat['Compliments'] + subcat['Questions'] + subcat['Suggestions']
cc_counts = cc.drop(['Product Code', 'itemid_4', 'elc_brand', "date"], axis = 1).columns

In [25]:
for col in cc_counts:
    full_relative_data["percent_" + col] = subcat[col]/subcat["cc_tot"]

In [26]:
full_relative_data = full_relative_data.fillna(0)

In [27]:
if BRAND == None:
    full_relative_data.to_csv(onedrive_path + "Data/Combined/relative_rr_cc_data_w_target_encodings.csv")
else:
    full_relative_data.to_csv(onedrive_path + "Data/Combined/relative_rr_cc_data_w_target_encodings_" + BRAND + ".csv")